In [ ]:
import scipy.io as sio
import os
import numpy as np
import pandas as pd
from scipy.signal import spectrogram
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report, make_scorer, accuracy_score, precision_score, recall_score, f1_score
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, StratifiedKFold, KFold, cross_validate, RepeatedKFold 
from sklearn.model_selection import train_test_split
from scipy import signal
import math

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from tensorflow.keras.utils import to_categorical 

from keras.models import Sequential, load_model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, Conv2D, MaxPooling2D, RepeatVector
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
os.getcwd()
os.chdir('/content/drive/My Drive')

In [ ]:
_,_, files = next(os.walk('/content/drive/My Drive/MTech_Thesis/DENS/Emotional'))
files_count = len(files)

In [ ]:
files

['Joyous_mit003Trial-2Click-1.mat',
 'Afraid_mit003Trial-6Click-2.mat',
 'Angry_mit003Trial-7Click-4.mat',
 'Hate_mit003Trial-7Click-3.mat',
 'Hate_mit003Trial-10Click-6.mat',
 'Alarmed_mit003Trial-9Click-5.mat',
 'Sad_mit003Trial-11Click-7.mat',
 'Amused_mit004Trial-3Click-1.mat',
 'Adventorous_mit004Trial-4Click-3.mat',
 'Excited_mit004Trial-4Click-2.mat',
 'Disgust_mit004Trial-5Click-4.mat',
 'Disgust_mit004Trial-5Click-5.mat',
 'Happy_mit004Trial-8Click-6.mat',
 'Happy_mit004Trial-8Click-7.mat',
 'Taken Aback_mit004Trial-11Click-9.mat',
 'Distress_mit007Trial-3Click-2.mat',
 'Miserable_mit007Trial-2Click-1.mat',
 'Despondent_mit007Trial-4Click-4.mat',
 'Miserable_mit007Trial-4Click-3.mat',
 'Happy_mit007Trial-7Click-6.mat',
 'Hate_mit007Trial-5Click-5.mat',
 'Joyous_mit007Trial-7Click-7.mat',
 'Calm_mit014Trial-1Click-1.mat',
 'Pensive_mit014Trial-1Click-3.mat',
 'Pensive_mit014Trial-1Click-2.mat',
 'Amused_mit014Trial-2Click-4.mat',
 'Excited_mit014Trial-2Click-5.mat',
 'Amused_mi

In [ ]:
rating = pd.read_csv('/content/drive/My Drive/MTech_Thesis/DENS/Rating/wholeFrequencyDependentDataWithVADLFR_ReFormattingWholeFrequencyVA - wholeFrequencyDependentDataWithVADLFR_ReFormattingWholeFrequencyVA.csv')

In [ ]:
eeg_data = []
eeg_label = []

In [ ]:
def label_mapping(valence, arousal):
    # HVHA, HVLA, LVHA, and LVLA
    
    if (valence > 5 and arousal > 5):
        label = 0 
    elif (valence > 5 and arousal <= 5):
        label = 1 
    elif (valence <= 5 and arousal > 5):
        label = 2 
    elif (valence <= 5 and arousal <= 5):
        label = 3
    
    return label

In [ ]:
base_path = "/content/drive/My Drive/MTech_Thesis/DENS/Emotional/"
for f in files:
  if f.endswith(".mat"):
    file_str = f[:-4]
    df = rating[rating["Subject"].str.contains(file_str)]
    if not df.empty:
      path = base_path + f
      mat = sio.loadmat(path)
      data = mat.get('eegData')
      eeg_data.append(data)
      df = df[['Subject','valence','arousal']]
      df = df.drop_duplicates()
      valence = df['valence']
      arousal = df['arousal']
      lab = label_mapping(valence.values[0], arousal.values[0])
      eeg_label.append(lab)

In [ ]:
eeg_data = np.array(eeg_data)

In [ ]:
eeg_data.shape

(465, 128, 1751)

In [ ]:
eeg_label = np.array(eeg_label)

In [ ]:
eeg_Label = []
for i in eeg_label:
    for j in range (0,128):
        eeg_Label.append(i)

In [ ]:
from tensorflow.keras.utils import to_categorical 
y = to_categorical(eeg_Label)

In [ ]:
import functools
def combine_dims(a, i=0, n=1):
  s = list(a.shape)
  combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
  return np.reshape(a, s[:i] + [combined] + s[i+n+1:])

In [ ]:
eeg_Data = combine_dims(np.array(eeg_data), 0)

In [ ]:
fs = 250

size_dataset = len(eeg_Data)
nperseg = 125
noverlap = 62
f_size = math.ceil(((nperseg + 1)/2))
t_size = int((eeg_Data[0].size - noverlap ) / (nperseg - noverlap))
X_full = np.ndarray(shape=(size_dataset, f_size, t_size, 3))                     
for i in range(0, size_dataset):
  for j in range(0,3):
    X = eeg_Data[i] 
    f, t, Sxx = spectrogram(X, fs, nperseg = nperseg, noverlap = noverlap, mode='psd')
    X_full[i, :, :, j] = Sxx

In [ ]:
X_full = np.array(X_full)

In [ ]:
X_full /= 255

In [ ]:
input_shape = (X_full.shape[1],X_full.shape[2],X_full.shape[3])
input_shape

(63, 26, 3)

In [ ]:
zero_weight = 0.95
one_weight = 8.94
two_weight = 0.48
three_weight = 1.35

In [ ]:
def build_Classifier():
  model = Sequential()

  model.add(Conv2D(32, (3, 3), 
            strides=(1, 1),
            activation='relu', 
            input_shape=input_shape))
  model.add(MaxPooling2D((2, 2), strides=(2, 2)))

  model.add(Conv2D(64, (3, 3), 
            activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))

  model.add(Flatten())

  model.add(RepeatVector(4))
  
  model.add(LSTM(256, return_sequences=True))
  model.add(Dropout(0.2))

  model.add(LSTM(128))
  model.add(Dropout(0.2))

  model.add(Dense(64, activation="relu"))
  model.add(Dropout(0.2))

  model.add(Dense(4, activation="softmax"))
  model.compile(optimizer ="Adam", loss =keras.losses.categorical_crossentropy, metrics=['accuracy'], weighted_metrics=["accuracy"])

  return model

In [ ]:
kf = RepeatedKFold(n_splits=5,n_repeats=5, random_state = 42)

In [ ]:
accuracies = []
weighted_accuracies = []
f_scores = []

In [26]:
for train_index, test_index in kf.split(X_full, y):
  X_train, X_test = X_full[train_index], X_full[test_index]
  y_train, y_test = y[train_index], y[test_index]

  model = build_Classifier()
  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
  mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
  m=model.fit(X_train, y_train,epochs=100,batch_size=256,verbose=1,validation_data=(X_test, y_test), class_weight={0:zero_weight, 1:one_weight, 2: two_weight, 3:three_weight}, callbacks =[es, mc])

  saved_model = load_model('best_model.h5')
  _, test_acc,weighted_test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
  accuracies.append(test_acc)
  weighted_accuracies.append(weighted_test_acc)
  y_pred_saved = saved_model.predict(X_test)
  y_test1=np.argmax(y_test, axis=1)
  y_pred_saved=np.argmax(y_pred_saved,axis=1)
  f_score = f1_score(y_test1,y_pred_saved, average = 'weighted')
  f_scores.append(f_score)

Streaming output truncated to the last 5000 lines.
Epoch 51/100
184/186 [============================>.] - ETA: 0s - loss: 0.0883 - accuracy: 0.9554 - weighted_accuracy: 0.9682
Epoch 51: val_accuracy did not improve from 0.94682
186/186 [==============================] - 4s 20ms/step - loss: 0.0882 - accuracy: 0.9554 - weighted_accuracy: 0.9682 - val_loss: 0.1728 - val_accuracy: 0.9409 - val_weighted_accuracy: 0.9409
Epoch 52/100
184/186 [============================>.] - ETA: 0s - loss: 0.0806 - accuracy: 0.9576 - weighted_accuracy: 0.9697
Epoch 52: val_accuracy did not improve from 0.94682
186/186 [==============================] - 4s 20ms/step - loss: 0.0805 - accuracy: 0.9576 - weighted_accuracy: 0.9697 - val_loss: 0.1920 - val_accuracy: 0.9358 - val_weighted_accuracy: 0.9358
Epoch 53/100
186/186 [==============================] - ETA: 0s - loss: 0.0712 - accuracy: 0.9631 - weighted_accuracy: 0.9740
Epoch 53: val_accuracy improved from 0.94682 to 0.94960, saving model to best_model

In [27]:
accuracies = np.array(accuracies)
weighted_accuracies = np.array(accuracies)
f_scores = np.array(f_scores)

In [28]:
print("Accuracy", accuracies)
print("Mean Accuracy", np.mean(accuracies))
print("Balanced Accuracy", weighted_accuracies)
print("Mean Balanced Accuracy", np.mean(weighted_accuracies))
print("F1 score", f_scores)
print("Mean F1 score", np.mean(f_scores))

Accuracy [0.96866602 0.96925402 0.96942204 0.96723789 0.9669019  0.96614581
 0.96891803 0.96875    0.96715391 0.96782595 0.96950603 0.97118616
 0.97110218 0.96572578 0.96639782 0.97009408 0.96782595 0.96975809
 0.96832997 0.96732193 0.96959007 0.97143817 0.96379369 0.96522176
 0.96967405]
Mean Accuracy 0.9682896518707276
Balanced Accuracy [0.96866602 0.96925402 0.96942204 0.96723789 0.9669019  0.96614581
 0.96891803 0.96875    0.96715391 0.96782595 0.96950603 0.97118616
 0.97110218 0.96572578 0.96639782 0.97009408 0.96782595 0.96975809
 0.96832997 0.96732193 0.96959007 0.97143817 0.96379369 0.96522176
 0.96967405]
Mean Balanced Accuracy 0.9682896518707276
F1 score [0.96859706 0.96921825 0.96935153 0.96722583 0.96692105 0.96609336
 0.96883859 0.96878654 0.96715381 0.96778242 0.96947428 0.97114759
 0.97107933 0.96566226 0.966363   0.9701017  0.96781888 0.96974618
 0.96828895 0.96730414 0.96955485 0.97143653 0.96381073 0.9652347
 0.969654  ]
Mean F1 score 0.968265823877907
